In [30]:
from neo4j import GraphDatabase

# === Configuration ===
NEO4J_URI = "neo4j+ssc://7bc3d0ce.databases.neo4j.io"  # or your cloud URI
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "5Yn5t-lOXiZ_PfsM1AN09HYoL9aTrwC-rdSHaQP8FHw"

In [31]:
from neo4j import GraphDatabase
import pandas as pd
import csv

class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
    
    def close(self):
        self.driver.close()
    
    def execute_query(self, query, parameters=None):
        with self.driver.session() as session:
            result = session.run(query, parameters)
            return result.data()

In [54]:
def create_nodes_from_csv(conn, csv_file, node_label, node_id_column):
    """Create nodes from CSV file"""
    df = pd.read_csv(csv_file)
    
    for _, row in df.iterrows():
        # Create node with all properties from CSV row
        properties = {k: v for k, v in row.items() if pd.notna(v)}
        
        query = f"""
        MERGE (n:{node_label} {{{node_id_column}: $id}})
        SET n += $properties
        """
        
        conn.execute_query(query, {
            'id': row[node_id_column],
            'properties': properties
        })

def create_relationships_from_csv(conn, csv_file, from_label, to_label, 
                                 from_id_col, to_id_col, rel_type):
    """Create relationships from CSV file"""
    try:
        df = pd.read_csv(csv_file)
    
        for _, row in df.iterrows():
            query = f"""
            MATCH (a:{from_label} {{{from_id_col}: $from_id}})
            MATCH (b:{to_label} {{{to_id_col}: $to_id}})
            CREATE (a)-[r:{rel_type}]->(b)
            """
            
            conn.execute_query(query, {
                'from_id': row[from_id_col],
                'to_id': row[to_id_col]
            })
    except Exception as e:
        print(e)

# Usage example
conn = Neo4jConnection(NEO4J_URI, "neo4j", NEO4J_PASSWORD)



In [34]:
# Create nodes from CSV
create_nodes_from_csv(conn, "knowledge_graph_tables/Product.csv", "Product", "Product_ID")
create_nodes_from_csv(conn, "knowledge_graph_tables/Location_Product.csv", "Location_Product", "Loc_Product_ID")


In [35]:
create_nodes_from_csv(conn, "knowledge_graph_tables/Location_Resource.csv", "Location_Resource", "Loc_Resource_ID")
create_nodes_from_csv(conn, "knowledge_graph_tables/Location_Source.csv", "Location_Source", "Location_ID")
create_nodes_from_csv(conn, "knowledge_graph_tables/Production_Source_Header.csv", "Production_Source_Header", "Source_ID")
create_nodes_from_csv(conn, "knowledge_graph_tables/Resource.csv", "Resource", "Resource_ID")
create_nodes_from_csv(conn, "knowledge_graph_tables/UoM_Conversion_Factor.csv", "UoM_Conversion_Factor", "Product_ID")

In [36]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Location_Source.csv", "Location_Source", "Product", 
                            "Location_ID",  "Product_ID", "LOCATION_SOURCE_SUPPLIES_TO_PRODUCT")

In [37]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Location_Source.csv", "Location_Source", "Location_Product", 
                            "Ship_From_Loc_ID",  "Product_ID", "LOCATION_SOURCE_APPLIES_TO_LOCATION_PRODUCT")

In [38]:
create_relationships_from_csv(conn, "knowledge_graph_tables/UoM_Conversion_Factor.csv", "UoM_Conversion_Factor", "Product", 
                             "Target_UoM", "Product_ID", "UOM_CONVERSION_FACTOR_RELATES_TO_PRODUCT")

In [39]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Production_Source_Header.csv", "Production_Source_Header", "Product", 
                             "Source_ID", "Product_ID", "PRODUCTION_SOURCE_HEADER_PRODUCES_FOR_PRODUCT")

In [40]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Location_Product.csv", "Location_Product", "Product", 
                            "Loc_Product_ID", "Product_ID",  "LOCATION_DETAILS_FOR_PRODUCT")

In [41]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Production_Source_Header.csv","Production_Source_Header", "Location_Source",  
                            "Source_ID", "Location_ID",  "PRODUCTION_SOURCE_HEADER_PRODUCES_LOCATION_PRODUCT")

In [42]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Location_Resource.csv","Location_Resource", "Location_Source",  
                            "Loc_Resource_ID", "Location_ID",  "LOCATION_RESOURCE_APPLIES_TO_LOCATION_SOURCE")

In [43]:
create_relationships_from_csv(conn, "knowledge_graph_tables/Location_Resource.csv","Location_Resource", "Resource",  
                            "Location_ID", "Resource_ID", "LOCATION_RESOURCE_USES_RESOURCE")

# Transactional data

In [44]:
create_nodes_from_csv(conn, "knowledge_graph_tables/DemandFulfillment.csv", "DemandFulfillment", "delivery_record_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/ReviewCapacity.csv", "ReviewCapacity", "location_resource_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/ReviewComponent.csv", "ReviewComponent", "product_location_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/ReviewDC.csv", "ReviewDC", "delivery_record_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/ReviewPlant.csv", "ReviewPlant", "product_location_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/ReviewVendors.csv", "ReviewVendors", "product_location_id")
create_nodes_from_csv(conn, "knowledge_graph_tables/ProfitMargin.csv", "ProfitMargin", "product_key_id")

In [53]:
create_relationships_from_csv(conn, "knowledge_graph_tables/ReviewVendors.csv","ReviewVendors", "ReviewPlant",  
                            "product_location_id", "LocationID", "VENDOR_SUPPLIES_RAW_MATERIALS")

In [46]:
create_relationships_from_csv(conn, "knowledge_graph_tables/ReviewComponent.csv","ReviewComponent", "ReviewPlant",  
                            "product_location_id", "LocationID", "RAW_MATERIAL_SUPPLY_TO_PLANT")

In [47]:
create_relationships_from_csv(conn, "knowledge_graph_tables/ReviewCapacity.csv","ReviewCapacity", "ReviewPlant",  
                            "location_resource_id", "LocationID", "RESOURCE_CAPACITY_FOR_PLANT")

In [65]:
create_relationships_from_csv_cleaned(conn, "knowledge_graph_tables/ReviewPlant.csv","ReviewPlant", "ReviewDC",  
                            "LocationID", "ProductID", "PLANT_SUPPLIES_PRODUCT_TO_DC")

Processing relationship: ReviewPlant(PL1000) -> ReviewDC(FG-1000)
Result: [{'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLIES_PRODUCT_TO_DC'}, {'type(r)': 'PLANT_SUPPLI

In [49]:
create_relationships_from_csv(conn, "knowledge_graph_tables/ReviewDC.csv","ReviewDC", "DemandFulfillment",  
                            "LocationID", "ProductID", "DC_SUPPLIES_PRODUCT_TO_CUSTOMER")

# relationship between transcational and master data

In [56]:
def create_relationships_from_csv_cleaned(conn, csv_file, from_label, to_label, 
                                  from_id_col, to_id_col, rel_type):
    """Create relationships from CSV file"""
    try:
        df = pd.read_csv(csv_file)
    
        for _, row in df.iterrows():
            print(f"Processing relationship: {from_label}({row[from_id_col]}) -> {to_label}({row[to_id_col]})")
            query = f"""
            MATCH (a:{from_label} {{{from_id_col}: $from_id}})
            MATCH (b:{to_label} {{{to_id_col}: $to_id}})
            CREATE (a)-[r:{rel_type}]->(b)
            RETURN type(r)
            """
            result = conn.execute_query(query, {
                'from_id': row[from_id_col],
                'to_id': row[to_id_col]
            })
            print(f"Result: {result}")
    except Exception as e:
        print(f"Error: {e}")


In [62]:
create_relationships_from_csv_cleaned(conn, "knowledge_graph_tables/ReviewPlant.csv","Product", "ReviewPlant",  
                            "ProductID","ProductID", "PRODUCT_DETAILS_FOR_PLANT")

Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []
Processing relationship: Product(FG-1000) -> ReviewPlant(FG-1000)
Result: []

In [64]:
create_relationships_from_csv_cleaned(conn, "knowledge_graph_tables/ReviewCapacity.csv","Resource", "ReviewCapacity",  
                            "ResourceID","LocationID", "RESOURCE_DETAILS")

Processing relationship: Resource(RES1000_001) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES1000_001) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES1000_001) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES1000_001) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES1000_002) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES1000_002) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES1000_002) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES1000_002) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES3000_001) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES3000_001) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES3000_001) -> ReviewCapacity(PL1000)
Result: []
Processing relationship: Resource(RES3000_001) -> ReviewCapacity(PL1000)
Res

In [67]:
create_relationships_from_csv_cleaned(conn, "knowledge_graph_tables/ProfitMargin.csv","Product", "ProfitMargin",  
                            "ProductID","KeyFigure", "PROFIT_MARGIN_FOR_PRODUCT")

Processing relationship: Product(FG-1000) -> ProfitMargin(Customer Receipts)
Result: []
Processing relationship: Product(FG-1000) -> ProfitMargin(Planned Price (Supply))
Result: []
Processing relationship: Product(FG-1000) -> ProfitMargin(Constrained Demand Revenue)
Result: []
Processing relationship: Product(FG-1000) -> ProfitMargin(Incoming Production Receipts)
Result: []
Processing relationship: Product(FG-1000) -> ProfitMargin(Production Cost Rate of Product)
Result: []
Processing relationship: Product(FG-1000) -> ProfitMargin(Final Supply Cost)
Result: []
Processing relationship: Product(FG-1000) -> ProfitMargin(Final Margin)
Result: []
Processing relationship: Product(FG-2000) -> ProfitMargin(Customer Receipts)
Result: []
Processing relationship: Product(FG-2000) -> ProfitMargin(Planned Price (Supply))
Result: []
Processing relationship: Product(FG-2000) -> ProfitMargin(Constrained Demand Revenue)
Result: []
Processing relationship: Product(FG-2000) -> ProfitMargin(Incoming Produ